 # Libraries

In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp

 # Keypoints using Mediapipe Holistic Model

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(frame, model):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Model (Holistic) works on RGB not BGR
#     frame.flags.writeable = False # To improve performance, optionally mark the image as not writeable to pass by reference
    results = model.process(frame)                 # Make prediction
#     frame.flags.writeable = True
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    return frame, results

In [4]:
def draw_landmarks(frame, model): # BGR
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80, 200, 100), thickness=2, circle_radius=3),  # Points
                             mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2) # Connection
                             ) 
    mp_drawing.draw_landmarks(frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=1, circle_radius=1),     
                             mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=1)  
                             )   
    mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=3),
                             mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2)
                             )    
    mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=3),
                             mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2)
                             )  

In [12]:
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER]

x: 0.711671
y: 0.5568175
z: -1.2877527
visibility: 0.882982

In [18]:
results.pose_landmarks.landmark[11:22]

[x: 0.77689016
 y: 0.57905567
 z: -1.3680867
 visibility: 0.8770314,
 x: 0.259682
 y: 0.65471905
 z: -1.4111807
 visibility: 0.9958197,
 x: 0.9277129
 y: 0.8930924
 z: -1.253278
 visibility: 0.29676133,
 x: 0.061193526
 y: 1.0089518
 z: -1.2454627
 visibility: 0.87377685,
 x: 0.9081963
 y: 1.126192
 z: -1.5884081
 visibility: 0.07053942,
 x: 0.2216248
 y: 1.3774383
 z: -1.4163444
 visibility: 0.433841,
 x: 0.91804945
 y: 1.1774061
 z: -1.6750755
 visibility: 0.05385783,
 x: 0.24441218
 y: 1.4717813
 z: -1.4658897
 visibility: 0.29870552,
 x: 0.8975908
 y: 1.1645185
 z: -1.6925739
 visibility: 0.06753253,
 x: 0.29194728
 y: 1.4232881
 z: -1.504106
 visibility: 0.36715922,
 x: 0.87636983
 y: 1.1609402
 z: -1.60068
 visibility: 0.07822117]

In [5]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic() as holistic:
    while True:
        ret, frame = cap.read()
#         frame = cv2.flip(frame,1)
        if ret == False:
            break

        frame, results = mediapipe_detection(frame, holistic)
        # print(results)  # <class 'mediapipe.python.solution_base.SolutionOutputs'>
        
        # Draw landmarks
        draw_landmarks(frame, results)
        
        cv2.imshow('Esm3ni', frame)
        key = cv2.waitKey(1)
        if key == 27 or key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [32]:
cap.release()
cv2.destroyAllWindows()

 # Extract Keypoints Values

In [27]:
print(len(results.face_landmarks.landmark))
len(results.face_landmarks.landmark)*3
# x, y, z

468


1404

In [31]:
print(len(results.right_hand_landmarks.landmark))
len(results.right_hand_landmarks.landmark)*3
# x, y, z

21


63

In [28]:
print(len(results.pose_landmarks.landmark))
len(results.pose_landmarks.landmark)*4
# x, y, z, visibility

33


132

In [40]:
print(len(results.pose_landmarks.landmark[11:23]))
len(results.pose_landmarks.landmark[11:23])*4
# x, y, z, visibility

12


48

In [34]:
def extract_keypoints(results):
    pose = np.array( [ [res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark ] ).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lefthand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    righthand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([pose, face, lefthand, righthand])

In [51]:
   132 + 1404 + 2 * 63
# pose + face + 2 * hand

1662

In [ ]:
def extract_keypoints(results):
    pose = np.array( [ [res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark[11:23] ] ).flatten() if results.pose_landmarks else np.zeros(48)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lefthand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    righthand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([pose, face, lefthand, righthand])

In [38]:
   48  + 1404 + 2 * 63
# pose + face + 2 * hand

1578

 # For Training

 # Create Folders for Actions

In [14]:
DATA_PATH = os.path.join('Data')

actions = np.array(['ds'])  # Words

no_videos = 30

video_length = 30  # Video is 30 frames in length

for action in actions:
    for video in range(no_videos):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(video)))
        except:
            print(action, video)

 # Create Data

In [48]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic() as holistic:
   
    for action in actions:
        for video in range(no_videos):
            for frame_num in range(video_length):       
                ret, frame = cap.read()
                frame = cv2.flip(frame,1) # Test
                if ret == False:
                    break

                frame, results = mediapipe_detection(frame, holistic)
                draw_landmarks(frame, results)
                
                # Wait between each video
                if frame_num == 0:                    
                    cv2.waitKey(2000)

                cv2.putText(frame, f'Collecting frames for {action} video number {video+1}', (15,20),\
                            cv2.FONT_ITALIC, 0.5, (0,0,255), 2)
                    
                # Export Keypoints (Save as numpy)
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(video), str(frame_num))
                np.save(npy_path, keypoints)
                
                cv2.imshow('Esm3ni', frame)
        
                key = cv2.waitKey(1)
                if key == 27 or key == ord('q'):
                    break 

    cap.release()
    cv2.destroyAllWindows()

In [46]:
cap.release()
cv2.destroyAllWindows()

 # Works on DL Model

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [16]:
actions = np.array(os.listdir(DATA_PATH))
actions

array(['excuse me', 'goodbye', 'hello', 'thanks'], dtype='<U9')

In [17]:
action_map = {action: idd for idd, action in enumerate(actions)}
action_map

{'excuse me': 0, 'goodbye': 1, 'hello': 2, 'thanks': 3}

In [18]:
videos, labels = [], []
for action in actions:
    for video in range(no_videos):
        temp = []
        for frame_num in range(video_length):
            res = np.load(os.path.join(DATA_PATH, action, str(video), f'{frame_num}.npy'))
            temp.append(res)
        videos.append(temp)
        labels.append(action_map[action])

In [19]:
x = np.array(videos)
y = to_categorical(labels, dtype='int')

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [51]:
y

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1,

In [63]:
print(x.shape, y.shape)

(60, 30, 1578) (60, 2)


In [272]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(96, 30, 1662) (96, 4)
(24, 30, 1662) (24, 4)


In [273]:
96 * 4

384

 # CNN

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

In [23]:
model = Sequential()
model.add(Flatten(input_shape=(30, 1662)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax')) # sigmoid

In [24]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [25]:
model.fit(x_train, y_train, epochs=100) #, callbacks=[tb_callback])

Epoch 1/100
3/3 [==============================] - 2s 227ms/step - loss: 41.8323 - categorical_accuracy: 0.2396
Epoch 2/100
3/3 [==============================] - 1s 240ms/step - loss: 32.0584 - categorical_accuracy: 0.2500
Epoch 3/100
3/3 [==============================] - 1s 252ms/step - loss: 18.1802 - categorical_accuracy: 0.2812
Epoch 4/100
3/3 [==============================] - 1s 249ms/step - loss: 17.4501 - categorical_accuracy: 0.2604
Epoch 5/100
3/3 [==============================] - 1s 246ms/step - loss: 7.4938 - categorical_accuracy: 0.3958
Epoch 6/100
3/3 [==============================] - 1s 247ms/step - loss: 5.3178 - categorical_accuracy: 0.4583
Epoch 7/100
3/3 [==============================] - 1s 232ms/step - loss: 2.8665 - categorical_accuracy: 0.6875
Epoch 8/100
3/3 [==============================] - 1s 224ms/step - loss: 2.7238 - categorical_accuracy: 0.5208
Epoch 9/100
3/3 [==============================] - 1s 234ms/step - loss: 1.0906 - categorical_accuracy: 0.67

3/3 [==============================] - 1s 239ms/step - loss: 0.0071 - categorical_accuracy: 1.0000
Epoch 75/100
3/3 [==============================] - 1s 238ms/step - loss: 0.0078 - categorical_accuracy: 1.0000
Epoch 76/100
3/3 [==============================] - 1s 236ms/step - loss: 0.0069 - categorical_accuracy: 1.0000
Epoch 77/100
3/3 [==============================] - 1s 240ms/step - loss: 0.0072 - categorical_accuracy: 1.0000
Epoch 78/100
3/3 [==============================] - 1s 242ms/step - loss: 0.0065 - categorical_accuracy: 1.0000
Epoch 79/100
3/3 [==============================] - 1s 235ms/step - loss: 0.0071 - categorical_accuracy: 1.0000
Epoch 80/100
3/3 [==============================] - 1s 241ms/step - loss: 0.0063 - categorical_accuracy: 1.0000
Epoch 81/100
3/3 [==============================] - 1s 235ms/step - loss: 0.0064 - categorical_accuracy: 1.0000
Epoch 82/100
3/3 [==============================] - 1s 242ms/step - loss: 0.0063 - categorical_accuracy: 1.0000
Epoch

In [18]:
print(model.evaluate(x_test, y_test))
model.evaluate(x_train, y_train)

1/1 [==============================] - 3s 3s/step - loss: 0.4381 - categorical_accuracy: 0.8750
[0.4381135404109955, 0.875]
3/3 [==============================] - 5s 110ms/step - loss: 0.1985 - categorical_accuracy: 0.9896


[0.1985018253326416, 0.9895833134651184]

 # Test in Real Time

In [37]:
video = []
sentence = []
threshold = 0.4

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic() as holistic:
    while True:
        ret, frame = cap.read()
        frame = cv2.flip(frame,1)
        if ret == False:
            break

        frame, results = mediapipe_detection(frame, holistic)
#         draw_landmarks(frame, results)
        
        keypoints = extract_keypoints(results)
        video.append(keypoints)
        video = video[-30:]
        
        if len(video) == 30:
            res = model.predict(np.expand_dims(video, axis=0))[0]

        if res[np.argmax(res)] > threshold:
            if len(sentence) == 0:
                sentence.append(actions[np.argmax(res)])
                
            if actions[np.argmax(res)] != sentence[-1]:
                sentence.append(actions[np.argmax(res)])
                
        if len(sentence) > 5:
            sentence = sentence[-5:]
            
        cv2.rectangle(frame, (0,0), (640,40), (255, 255, 255), -1)
        cv2.putText(frame, ' '.join(sentence), (3,30), cv2.FONT_ITALIC, 1, (0,0,0), 2, cv2.LINE_AA)
        print(sentence[-1])
        
        cv2.imshow('OpenCV', frame)
        key = cv2.waitKey(1)
        if key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
thanks
excuse me
thanks
excuse me
excuse me
excuse me
excuse me
excuse me
excuse me
excuse me
excuse me
excuse me
excuse me
excuse me
excuse me
excuse me
excuse me
excuse me
excuse me
excuse me
excuse me
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
goodbye
hello
goodbye
hello
goodbye
goodbye
goodbye
goodbye
hello
hello
hello
hello
hello


In [27]:
cap.release()
cv2.destroyAllWindows()

In [36]:
sentence

['hello',
 'goodbye',
 'thanks',
 'excuse me',
 'goodbye',
 'hello',
 'goodbye',
 'hello',
 'goodbye',
 'hello',
 'goodbye',
 'hello',
 'goodbye',
 'hello']

 # Save & Load Model

In [174]:
model.save('test.h5')

In [176]:
from keras.models import load_model
loaded_model = load_model('test.h5')
loss, accuracy = loaded_model.evaluate(x_test, y_test)

1/1 [==============================] - 2s 2s/step - loss: 9.9341e-09 - categorical_accuracy: 1.0000
